In [1]:
import ray

In [2]:
ray.shutdown()

In [2]:
ray.init()

2021-05-04 16:17:33,432	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8266


{'node_ip_address': '9.160.24.40',
 'raylet_ip_address': '9.160.24.40',
 'redis_address': '9.160.24.40:29280',
 'object_store_address': '/tmp/ray/session_2021-05-04_16-17-32_831080_96327/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-05-04_16-17-32_831080_96327/sockets/raylet',
 'webui_url': '127.0.0.1:8266',
 'session_dir': '/tmp/ray/session_2021-05-04_16-17-32_831080_96327',
 'metrics_export_port': 58844,
 'node_id': 'c8ed8ccd822336e1bcfc1d946811186c9b954ceb'}

In [3]:
%config IPCompleter.use_jedi = False

In [4]:
import numpy as np
from sklearn.preprocessing import FunctionTransformer

transformer = FunctionTransformer(np.log1p)

In [6]:
import com.ibm.research.ray.graph.Datamodel as dm

pipeline = dm.Pipeline()

node_a = dm.OrNode('a', transformer)
node_b = dm.OrNode('b', transformer)
node_c = dm.OrNode('c', transformer)
node_d = dm.OrNode('d', transformer)

pipeline.add_edge(node_a, node_b)
pipeline.add_edge(node_b, node_c)
# pipeline.add_edge(node_b, node_d)

In [7]:
X = np.array([0, 1, 2, 3])

In [8]:
transformer.transform(transformer.transform(transformer.transform(X)))

array([0.        , 0.42303586, 0.55461836, 0.62580029])

In [9]:
from com.ibm.research.ray.graph.Datamodel import XY
from com.ibm.research.ray.graph.Datamodel import XYRef

In [10]:
# Xy = XY(X, None)
# Xy_ref = ray.put(Xy)

X_ref = ray.put(X)
y_ref = ray.put(None)

Xy_ref = XYRef(X_ref, y_ref)
Xy_ref_list = [Xy_ref]
Xy_ref_ptr = ray.put(Xy_ref_list)

In [11]:
import com.ibm.research.ray.graph.Runtime as rt

In [12]:
t = [Xy_ref_ptr]

In [13]:
in_args={node_a: t}

In [14]:
from com.ibm.research.ray.graph.Runtime import ExecutionType

In [15]:
out_args = rt.execute_pipeline(pipeline, ExecutionType.TRAIN, in_args)

In [16]:
for key in out_args.keys():
    print(str(key))

c


In [17]:
ray.get(out_args[node_c])

[[<com.ibm.research.ray.graph.Datamodel.XYRef at 0x7f9898928190>]]